# Clustering

The goal of this program is to visualize and cluster the data.

## Input
* Importing the libaries that are necessary:

In [1]:
import numpy as np
import pandas as pd

* Importing the necessary files:
 * Read files
 * Convert read files to data
 * Save data in dataframes

In [38]:
df_data = pd.read_csv('Data\Voorbeeld_clusterdata.txt', sep='  ', header=None, engine='python', index_col=0)
df_results = pd.read_csv('Data\Voorbeeld_clusterresult.txt', sep='  ', header=None, engine='python', index_col=0)
df_clusterd = df_data.copy().drop(columns=df_data.columns)

def lib(FileName):
    def openFile(FileName):
        read_data = open(f'{FileName}')
        data = read_data.read()
        data = data.splitlines()
        read_data.close()
        
        return data
        
    lib = {}
    
    for lines in openFile(FileName):
        line = lines.split()
        lib[line[0]] = line[1:]
            
    return lib

lib_data = lib('Data\Voorbeeld_clusterdata.txt')
lib_results = lib('Data\Voorbeeld_clusterresult.txt')

## Cluster methods
### K-means (KMCA):

uitleggen hoe het werkt

In [3]:
class KMCA:
    def __init__(self, k=6, tol=0.001, max_iter=300):
        self.k = k
        self.tol = tol
        self.max_iter = max_iter
        
    def set_k(K):
        self.k = K

    def fit(self,data):

        self.centroids = {}

        for i in range(self.k):
            self.centroids[i] = data[i]

        for i in range(self.max_iter):
            self.classifications = {}

            for i in range(self.k):
                self.classifications[i] = []

            for featureset in data:
                distances = [np.linalg.norm(featureset-self.centroids[centroid]) for centroid in self.centroids]
                classification = distances.index(min(distances))
                self.classifications[classification].append(featureset)

            prev_centroids = dict(self.centroids)

            for classification in self.classifications:
                self.centroids[classification] = np.average(self.classifications[classification],axis=0)

            optimized = True

            for c in self.centroids:
                original_centroid = prev_centroids[c]
                current_centroid = self.centroids[c]
                if np.sum((current_centroid-original_centroid)/original_centroid*100.0) > self.tol:
                    optimized = False

            if optimized:
                break

    def predict(self,data):
        def pred(self, data):
            distances = [np.linalg.norm(data-self.centroids[centroid]) for centroid in self.centroids]
            classification = distances.index(min(distances))
            return classification
        
        self.pred_classifications = list()
        
        if np.isfortran(data):
            for element in data:
                self.pred_classifications.append(pred(self, element))
        else:
            self.pred_classifications.append(pred(self, data))
            
        return self.pred_classifications

### Own cluster method: Grid based Cluster Algorithm (GCA)
VERDERE UITLEG MOET NOG KOMEN

In [95]:
class GCA:
    """
    THIS CLASS USES Dictionaries as input
    """
    
    def __init__(self, r=1, j=10, tresh=4):
        self.r = r
        self.j = j
        self.tresh = tresh
        
    def set_range(self, r):
        self.r = r
        
    def set_interval(self,j):
        self.j = j
    
    
    def fit(self, data):
        self.indexes = list(data.keys())

        def get_cordinates(data):
            cordinates = dict()

            for cor in range(len(data[self.indexes[0]])):
                x = list()
                for i in data:
                    x.append(float(data[i][cor]))

                cordinates[cor] = tuple(x)
            return cordinates

        def get_intervals(data,j):
            dic = dict()

            for i in data:
                minim = np.min(data[i])
                maxim = np.max(data[i])

                dic[i] = np.mgrid[minim:maxim:complex(0,j)]

            return dic

        def get_cells():
            cordinates = get_cordinates(data)
            intervals = get_intervals(cordinates,self.j)
            
            labels = list()
            for i in cordinates:
                x_inter_label = list()
                for j in range(1,len(intervals[i])):
                    vals = list((cordinates[i] >= intervals[i][j-1]) & (cordinates[i] < intervals[i][j]))
                    vals = np.where(vals==False, 0, vals)
                    vals = np.where(vals==True, j, vals)       

                    x_inter_label.append(vals)

                labels.append(x_inter_label)


            labelzz = list()
            for i in range(len(labels)):
                for k in range(1,len(labels[i])):
                    labels[i][0] = np.add(labels[i][0],labels[i][k])
                labelzz.append(labels[i][0])

            del labels

            self.lib_cells = dict()     

            for i in range(len(labelzz[0])):
                cell_id = list()
                for j in range(len(labelzz)):
                    cell_id.append(labelzz[j][i])

                self.lib_cells[self.indexes[i]] = cell_id
            
            
            return self.lib_cells
        return get_cells()
        
    def self_predict(self, cell_ids):
        """
        DIT STUK MOET NOG EFFICIENTER WORDEN GEMAAKT

        """
        def get_neigb_id(ID):
            neigb = list()
            for indx in cell_ids:
                distance = np.subtract(self.lib_cells[ID],self.lib_cells[indx],out=np.array([0 for i in range(8)]))
                if all([-self.r < i < self.r for i in distance]):
                    neigb.append(indx)

            return neigb

        new_indexes = self.indexes.copy()

        def make_groups(data_point):
            neigbors = get_neigb_id(data_point)
            new_neigbors = list()

            for cel in neigbors:
                new_cells = get_neigb_id(cel)

                for new_cel in new_cells:
                    if new_cel not in neigbors:
                        new_neigbors.append(new_cel)
                        print(new_cel)

                neigbors += new_neigbors

            for neigb in neigbors:
                new_indexes.remove(neigb)

            return neigbors

        options = list()

        for index in new_indexes:
            options.append(make_groups(index))


        self.labeled_data = dict()
        cluster = 0
        anti_cluster = -1

        for i in range(len(options)):
            if len(options[i]) > self.tresh:
                self.labeled_data[cluster] = options[i]
                cluster += 1
            else:
                self.labeled_data[anti_cluster] = options[i]
                anti_cluster -= 1

        return self.labeled_data
            
    def predict(self, data):
        """
        VERZIN CODE DIE KIJKT WELKE CLUSTER DMV VAN CEL-ID GATHERING HET DICHTSTE BIJ IS

        """
        

In [96]:
gca = GCA()
fitted_data = gca.fit(lib_data)

In [97]:
pd.DataFrame(fitted_data).transpose()

,0,1,2,3,4,5,6,7
846160,7,6,7,7,6,8,8,8
820434,6,5,6,7,5,6,7,7
849103,6,4,6,7,5,7,7,7
846353,8,6,7,8,7,8,8,8
848613,6,5,6,7,5,7,7,7
...,...,...,...,...,...,...,...,...
760857,8,7,8,8,8,8,9,9
805218,8,6,7,8,7,7,8,8
945014,6,5,6,7,6,7,7,7
805835,7,6,7,7,7,8,8,8


In [98]:
results = gca.self_predict(fitted_data)

In [108]:
results_formatted = dict()

for i in results:
    for ID in results[i]:
        results_formatted[ID] = i

pd.DataFrame(results_formatted, index=[0]).transpose()

,0
846160,0
1382381,0
657946,0
350787,0
1109733,0
...,...
403071,-139
334876,-140
387455,-141
793020,-141


## Output

* get cluster labels
* save labels in text file

### get cluster labels:

In [5]:
kmca = KMCA()
kmca.fit(df_data.values)
df_clusterd['KMCA'] = kmca.predict(df_data.values)

df_clusterd.head()

,KMCA
0,
846160,0
820434,4
849103,2
846353,3
848613,2


### save results:

In [6]:
df_clusterd.to_csv(r'Data\clusterresultaten.txt', header=None, index=True, sep=' ')